In [256]:
%reset
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
import gc
import os
import copy
import pickle
from os.path import exists
import mne
import numpy as np
from mne import create_info
from IPython.utils import io
import yasa
import seaborn as sns
import pandas as pd
import matlab
import matlab.engine
import os

gc.collect()

11

In [257]:
pkl_dir = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\\imported_eventDetectionChan\\filter0.1"
import_type = "eventDetectionChan"
output_dir_name = 'eventDetection\\done_on_imported'

In [258]:
import_path = f'{pkl_dir}\\{import_type}.pkl'

with open(import_path, "rb") as file:
    [allsubsdata_perFile, configu] = pickle.load(file)

fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
if not os.path.exists(fig_output_dir):
    os.mkdir(fig_output_dir)

configu['electrodes_names']

One file yasa detection 

In [260]:
# sub = '32'
# datafile = 2
# id = f"{sub}_{datafile}"

# datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
# datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)

# exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
# sp = yasa.spindles_detect(datafile_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,include=[2, 3],ch_names=configu['electrodes_names'],multi_only=True)
# print(sp)
# print(sp.summary())
# # sp.plot_average(center='Peak',time_before = 1,time_after = 1)
# # plt.show()
# #%matplotlib widget
# #sp.plot_detection()

In [261]:
# %matplotlib inline
# sp.plot_average(center='Peak', time_before=0.1, time_after=0.1);

Batch yasa detection and save

In [262]:
# yasa_output_dir = f"{configu['outputs_dir_path']}/{output_dir_name}/yasa"
# if not os.path.exists(yasa_output_dir):
#     os.mkdir(yasa_output_dir)

# for id in allsubsdata_perFile:
#     datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
#     datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
#     if 2 not in datafile_scoring or 3 not in datafile_scoring:
#         continue

#     exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=False)
#     sp = yasa.spindles_detect(datafile_data,sf=configu['sample_freq'], hypno = exmp_scoring_upsampled,include=[2, 3],ch_names=configu['electrodes_names'],multi_only=True)
#     #summary_df  = sp.summary()
#     summary_df.to_csv(f"{yasa_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_yasaSpindles.csv")
#     sp.plot_average(center='Peak', time_before=1, time_after=1)
#     plt.savefig(f"{yasa_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_yasaSpindles_average.png")

One file Nir&Andrillon Detection

In [263]:
# import matlab
# import matlab.engine

# sub = '32'
# datafile = 2 
# id = f"{sub}_{datafile}"

# datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
# datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
# datafile_data_cz = datafile_data[1,:]
# exmp_scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data_cz, sf_data=configu['sample_freq'], verbose=True)

# eng = matlab.engine.start_matlab()
# eng.SpindlesDetectionAlgorithm_fieldtrip(np.array(datafile_data_cz), configu['sample_freq'], np.array(exmp_scoring_upsampled))

# #spindles, spindlesOutEnergy, spindlesOutDuration, rej_cand = eng.SpindlesDetectionAlgorithm_fieldtrip(datafile_data_cz, configu['sample_freq'], exmp_scoring_upsampled)

# eng.quit()

Batch Nir&Andrillon Detection

In [264]:
def deleteAuxANFiles(andriNir_aux_output_dir):
    all_files_in_outputDir = os.listdir(andriNir_aux_output_dir)
    for file_in_outputDir in all_files_in_outputDir:
            os.remove(os.path.join( andriNir_aux_output_dir, file_in_outputDir ))


andriNir_code_dir = "C:\\Users\\User\\Cloud-Drive\\AnatArzi\\scripts\\Matlab_scripts - Sharons\\Andrillon_Nir_Algo"
andriNir_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir"
andriNir_aux_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir\\aux_mats"
spindles_output_columns =  ['spindleStartTime', 'spindleEndTime', 'peakTime', 'peakEnergy', 'peakEnergyNorm', 'freqSpindle', 'spindleDuration/SR', 'PowerSP', 'PowerAlpha', 'currentStage']
electrode_column_name = 'electrode'

if not os.path.exists(andriNir_output_dir):
    os.mkdir(andriNir_output_dir)

electrodes_names_eventDetect = ['F3','F4','C3','C4','P3','P4','O1','O2']

all_electodes_ss_key = 'SS_algo-AN_all'

for id in allsubsdata_perFile:
    datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
    datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
    scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
    ss_allElectrodes = pd.DataFrame()

    for electd_i, electrode_name_eventDetect in enumerate(electrodes_names_eventDetect):
        curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name_eventDetect)[0][0]

        ## create aux files to use in MATLAB
        datafile_1elect_eeg = datafile_data[curr_electrode_num,:]
        if 2 not in datafile_scoring or 3 not in datafile_scoring:
            continue
        
        mat_to_save =  {'datafile_data': datafile_1elect_eeg, 'scoring_upsampled': scoring_upsampled, 'sample_freq': configu['sample_freq'], 'electrode_name':electrode_name_eventDetect}
        scipy.io.savemat(f"{andriNir_aux_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat",mat_to_save)

        ## run Andrillon & Nir SS detection over all subjects files
        eng = matlab.engine.start_matlab()
        eng.cd(andriNir_code_dir, nargout=0)
        out = eng.batch_useAndrillonNirEventDetection(andriNir_aux_output_dir, andriNir_output_dir,nargout=0)
        eng.quit()

        ## add the spindles data to the main subject dictionary
        try: 
            spindles_file_name = f"{andriNir_output_dir}\\Spindles_andrillonNir_{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat"
            matlabImport = scipy.io.loadmat(spindles_file_name, simplify_cells=True)
        except Exception: 
            print(f"Error importing spindles sub file at: {allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}")
            continue

        ss = matlabImport['spindles']
        if np.size(ss) >0 :
            df = pd.DataFrame(np.double(ss))
            df.columns = spindles_output_columns
            tile_electrode = np.tile(f"{electrode_name_eventDetect}", ss.shape[0])[None].T
            df[electrode_column_name] = tile_electrode
            
            allsubsdata_perFile[id][f'SS_algo-AN@@{electrode_name_eventDetect}'] = df
            if electd_i ==0: ss_allElectrodes =  df
            else:  ss_allElectrodes = pd.concat([ss_allElectrodes,df])

        # sw = matlabImport['allWaves']
        # if np.size(sw) >0 :
        #     tile_electrode = np.tile(f"{electrode_name_eventDetect}", sw.shape[0])[None].T
        #     sw = np.hstack([sw, tile_electrode])
        #     allsubsdata_perFile[id][f'SW_algo-AN@@{electrode_name_eventDetect}'] = np.asarray(sw,dtype=object)  
        #     sw_allElectrodes = np.concatenate((sw_allElectrodes,sw))
 
    if not ss_allElectrodes.empty:
        allsubsdata_perFile[id][all_electodes_ss_key]  = ss_allElectrodes
        #allsubsdata_perFile[id][f'SW_algo-AN_all']  = sw_allElectrodes
    
deleteAuxANFiles(andriNir_aux_output_dir)

31-ינו-23 19:52:21 | WARNING | Hypnogram is SHORTER than data by 4.94 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 19:53:02 | WARNING | Hypnogram is SHORTER than data by 26.41 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 19:53:40 | WARNING | Hypnogram is SHORTER than data by 21.62 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 19:53:40 | WARNING | Hypnogram is SHORTER than data by 25.70 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 19:54:16 | WARNING | Hypnogram is SHORTER than data by 2.56 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 19:54:54 | WARNING | Hypnogram is SHORTER than data by 3.10 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 19:54:54 | WARNING | Hypnogram is SHORTER than data by 5.26 seconds. Padding hypnogram with last value to match data.size.
31-ינו-23 19:54:54 | WARNING | Hypnogram is SHORTER than data by 2

In [268]:
after_n2_exclution_key = f"{all_electodes_ss_key}_n2"
for id in allsubsdata_perFile:
    if all_electodes_ss_key not in allsubsdata_perFile[id]: continue
    ss_allElectrodes = allsubsdata_perFile[id][all_electodes_ss_key]
    ss_allElectrodes_n2 = ss_allElectrodes.loc[ss_allElectrodes['currentStage'] == 2.0]
    if len(ss_allElectrodes_n2) == 0: 
        continue
    else:
        print(f"sub:{id}, before excludeNonN2Events:{len(ss_allElectrodes)}, after:{len(ss_allElectrodes_n2)}")
    allsubsdata_perFile[id][after_n2_exclution_key] = ss_allElectrodes_n2

sub:32_1, before excludeNonN2Events:2419, after:789
sub:32_2, before excludeNonN2Events:2231, after:765
sub:33_1, before excludeNonN2Events:2491, after:1530
sub:33_2, before excludeNonN2Events:3556, after:2758
sub:34_1, before excludeNonN2Events:1363, after:416
sub:34_2, before excludeNonN2Events:1667, after:443
sub:35_1, before excludeNonN2Events:1076, after:190
sub:35_3, before excludeNonN2Events:1742, after:951
sub:35_4, before excludeNonN2Events:668, after:250
sub:38_1, before excludeNonN2Events:2531, after:1379
sub:38_2, before excludeNonN2Events:3091, after:1793
sub:38_3, before excludeNonN2Events:2127, after:1433


In [269]:
## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful

ss_key_to_use = all_electodes_ss_key # after_n2_exclution_key
uniqeElctd_ss_key = f"{ss_key_to_use}_uniqeElctd"

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

for id in allsubsdata_perFile:
    if ss_key_to_use not in allsubsdata_perFile[id]:  continue
    filterd_events_allElectrodes = allsubsdata_perFile[id][ss_key_to_use].copy(deep=True)
    
    filterd_events_allElectrodes.sort_values(by=['spindleStartTime'],inplace=True) 
    deleted = filterd_events_allElectrodes.copy(deep=True)
    filtered = pd.DataFrame([], columns = deleted.columns)
    simultan = pd.DataFrame([], columns = deleted.columns)
    while len(deleted)>0:
        if len(simultan)==0:
            simultan = pd.concat([simultan, deleted.iloc[[0]]])
            deleted.drop(deleted.index[0], axis=0, inplace=True)
            still_overlap = True
        else:
            while still_overlap == True:
                still_overlap = False
                simultan.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows

                ## check now_overlap_all_in_simultan:
                for index, simultan_row in simultan.iterrows():
                    simultan_0 = simultan_row['spindleStartTime']
                    simultan_1 = simultan_row['spindleEndTime']
                    deleted_0 = deleted.iloc[0]['spindleStartTime']
                    deleted_1 = deleted.iloc[0]['spindleEndTime']
                    if overlap([simultan_0,simultan_1],[deleted_0,deleted_1]):
                        simultan = pd.concat([simultan, deleted.iloc[[0]]])
                        deleted.drop(deleted.index[0], axis=0, inplace=True)
                        still_overlap = True
                        break
                if still_overlap: continue
                else:
                    ## check max ps and add to filt
                    simultan.sort_values(by=['PowerSP'],inplace=True)
                    row_df = simultan.iloc[[0]]
                    row_df.iat[0, row_df.columns.get_loc(electrode_column_name)] = np.array(simultan[electrode_column_name])
                    filtered = pd.concat([filtered,row_df])
                    simultan = pd.DataFrame([], columns = deleted.columns)                       
        
    filtered.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows
    allsubsdata_perFile[id][uniqeElctd_ss_key] = filtered   
    print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}")

sub:32_1, before filt:2419, after:532
sub:32_2, before filt:2231, after:417
sub:33_1, before filt:2491, after:677
sub:33_2, before filt:3556, after:711
sub:34_1, before filt:1363, after:350
sub:34_2, before filt:1667, after:238
sub:35_1, before filt:1076, after:395
sub:35_3, before filt:1742, after:427
sub:35_4, before filt:668, after:151
sub:38_1, before filt:2531, after:491
sub:38_2, before filt:3091, after:536
sub:38_3, before filt:2127, after:395


In [ ]:
from IPython.core.display import HTML
#display(HTML(allsubsdata_perFile[id][uniqeElctd_ss_key].to_html()))

Change scoring to fit EDF_viewer events format

In [270]:
def format_scoring_to_edfViewer(score):
    if score == 0:
        return 'W'
    elif score ==1:
        return 'N1'
    elif score ==2:
        return 'N2'
    elif score ==3:
        return 'N3'
    elif score ==4:
        return 'TREM'
    elif score ==5:
        return 'PREM'
    elif score ==6:
        return 'MOVE'
    elif score ==7:
        return 'ARTIFACT'
    else:
        Exception('no such score')
        

for id in allsubsdata_perFile:
    curr_file_scoring = allsubsdata_perFile[id]['scoring']
    new_format_score = np.zeros((len(curr_file_scoring),3), dtype=object)
    for ind, score in enumerate(curr_file_scoring):
            new_format_score[ind,:] = [30*ind,30,format_scoring_to_edfViewer(score)] ## onset (sec), duration, desc

    allsubsdata_perFile[id]['scoring_efdViewFormat'] = new_format_score
    # np.savetxt('test2.txt', new_format_score, delimiter='\t',fmt='%s')

Change spindles events to fit EDF_viewer events format

In [328]:
event_key_for_save = uniqeElctd_ss_key
SS_efdViewFormat_key = 'SS_efdViewFormat'
header = np.array(['Onset',"Duration","Annotation"])
for id in allsubsdata_perFile:
        if event_key_for_save not in allsubsdata_perFile[id]:
                print(f"no {event_key_for_save} for sub {id}")
                continue
        ss_df = allsubsdata_perFile[id][uniqeElctd_ss_key]
        ss_mat_edfFormat = np.zeros((len(ss_df),3), dtype=object)
        startTime_arr = np.array(ss_df['spindleStartTime']) / np.double(configu['sample_freq'])
        endTime_arr = np.array(ss_df['spindleEndTime']) / np.double(configu['sample_freq'])
        duration_arr = endTime_arr - startTime_arr
        electd_arr_per_ss = np.array(ss_df[electrode_column_name])
        desc = [f"SS@@{electd_arr[0]}" for electd_arr in electd_arr_per_ss]
        new_format_ss = np.array([startTime_arr,duration_arr,desc]).T ## onset (sec), duration,desc
        #new_format_ss = np.concatenate((new_format_ss,header))
        #new_format_ss = np.vstack(new_format_ss, header)
        new_format_ss = np.r_[new_format_ss,[header]]
        allsubsdata_perFile[id][f'SS_efdViewFormat'] = new_format_ss

no SS_algo-AN_all_uniqeElctd for sub 32_3
no SS_algo-AN_all_uniqeElctd for sub 33_3
no SS_algo-AN_all_uniqeElctd for sub 33_4
no SS_algo-AN_all_uniqeElctd for sub 34_3


In [ ]:
# # [spindleStartTime spindleEndTime peakTime peakEnergy peakEnergyNorm freqSpindle spindleDuration/SR PowerSP PowerAlpha currentStage];
# def EDFViewerformat(key_name):
#         curr_file_ss = allsubsdata_perFile[id][key_name]
#         new_format_ss = np.zeros((np.shape(curr_file_ss)[0],3), dtype=object)
#         for ind, ss in enumerate(new_format_ss):
#                 SpindleStartTime = np.double(curr_file_ss[ind,0]) / np.double(configu['sample_freq'])  
#                 SpindleDuration = (np.double(curr_file_ss[ind,1])  -np.double(curr_file_ss[ind,0])) / np.double(configu['sample_freq'])
#                 if np.size(curr_file_ss[ind,10]) > 1:  
#                         event_desc = f"{key_name.split('_')[0]}@@{curr_file_ss[ind,10][0]}"
#                 else:
#                         event_desc = f"{key_name.split('_')[0]}@@{curr_file_ss[ind,10]}"
#                 new_format_ss[ind,:] = [SpindleStartTime,SpindleDuration,event_desc] ## onset (sec), duration,desc
#         allsubsdata_perFile[id][f'{key_name}_efdViewFormat'] = new_format_ss



# events_types_for_save = [uniqeElctd_ss_key]
# for event_type in events_types_for_save: 
#         for id in allsubsdata_perFile:
#                 if event_type not in allsubsdata_perFile[id]:
#                         print(f"no {event_type} for sub {id}")
#                         continue
#                 EDFViewerformat(event_type)

In [331]:
edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events'
if not os.path.exists(edfViewFormat_events_output_dir):
    os.mkdir(edfViewFormat_events_output_dir)

events_types_for_save = ['scoring_efdViewFormat','SS']
# Save events (scoring and ss) per file, in a format suitable for EDF_viewer
for id in allsubsdata_perFile:
        filename = f"{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_events"
        all_events_with_header = np.asarray([['Onset',"Duration","Annotation"]],dtype=object)
        for event_type in events_types_for_save:
                events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if (event_type in k) and ('_efdViewFormat' in k)]
                if np.size(events_type_found) == 0: continue
                for events_found in events_type_found:
                        for event_found in events_found:
                                all_events_with_header = np.concatenate((all_events_with_header,[event_found]),dtype=object)
        np.savetxt(f"{edfViewFormat_events_output_dir}\\{filename}.txt", all_events_with_header, delimiter='\t',fmt='%s')

Test TP / FP for the algorithm of Andrillon-Nir

In [ ]:
## get the array of before manual scanning
txt_file_name = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\eventDetection\\done_on_preProcessed\\EDFViewFormat_events\\32_2_events.txt"
before = np.loadtxt(txt_file_name, delimiter="\t",dtype='object')
print(np.shape(before))
before[1:,0] = [np.double(x) for x in before[1:,0]]
before = np.delete(before,0,0)
ss_ind = np.array([],dtype=int)
for ind_i, desc in enumerate(before[:,2]):
    if "SS" in desc:
        ss_ind = np.append(ss_ind, ind_i)
before_ss = before[ss_ind,:]
print(np.shape(before_ss))

## get the array of after manual scanning
txt_file_name = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\eventDetection\\done_on_preProcessed\\EDFViewFormat_events\\s_32_sleep2_preProcessed_annotations.txt"
after = np.loadtxt(txt_file_name, delimiter="\t",dtype='object')
print(np.shape(after))
after[1:,0] = [np.double(x) for x in after[1:,0]]

ss_ind = np.array([],dtype=int)
for ind_i, desc in enumerate(after[:,2]):
    if "SS" in desc:
        ss_ind = np.append(ss_ind, ind_i)
after_ss = after[ss_ind,:]
print(np.shape(after_ss))

## compare to find rate od TP and FP
TP = 0
FP = 0
for ss_auto in before_ss:
    found = False
    for ss_manu in after_ss:
        if ss_auto[0] == ss_manu[0]:
            TP +=1
            found = True
            break
    if found == False:
        FP +=1

print(TP)
print(FP)

(510, 3)
(204, 3)
(737, 3)
(235, 3)
156
48


In [ ]:
## how to df 

## add row to df and empty df
x = pd.DataFrame([], columns = deleted.columns)
y= deleted.iloc[[0]]
y2= deleted.iloc[[1]]
z = pd.concat([x,y])
zz = pd.concat([z,y2])
display(zz)

## delete first row
# zz.drop(zz.index[0], axis=0, inplace=True) 
# display(zz)

## get value at i,j
print(deleted.iloc[0]['spindleStartTime'])

## get column as array
arr = np.array(zz['electrode'])
print(arr)

## replace inplace value in i,j to array:
zz.iat[1, zz.columns.get_loc(electrode_column_name)] = arr
zz